# Crawling data from internet

In [ ]:
import requests
import time
from tqdm import tqdm

In [160]:
import pymorphy2
import re
from nltk.corpus import stopwords
import numpy as np

## Itunes clawler

In [146]:
from lxml import html
import requests

/Users/maksim/anaconda/envs/py3k/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [3]:
class AppCrawler:
    def __init__(self, starting_url, depth):
        self.starting_url = starting_url
        self.depth = depth
        self.current_depth = 0
        self.depth_links = []
        self.apps = []
        
    def crawl(self):
        app = self.get_app_from_link(self.starting_url)
        self.apps.append(app)
        self.depth_links.append(app.links)
        while self.current_depth < self.depth:
            current_links = []
            for link in self.depth_links[self.current_depth]:
                current_app = self.get_app_from_link(link)
                current_links.extend(current_app.links)
                self.apps.append(current_app)
                time.sleep(5) #sleep for 5 secs
            self.current_depth += 1
            self.depth_links.append(current_links)
    
    def get_app_from_link(self, link):
        start_page = requests.get(link)
        #print(start_page.text)
        tree = html.fromstring(start_page.text)
        name = tree.xpath('//h1[@itemprop="name"]/text()')[0]
        developer = tree.xpath('//div[@class="left"]/h2/text()')[0]
        price = tree.xpath('//div[@itemprop="price"]/text()')[0]
        links = tree.xpath('//div[@class="center-stack"]//*/a[@class="name"]/@href')

        app = App(name, developer, price, links)
        return app

In [4]:
class App:
    def __init__(self, name, developer, price, links):
        self.name = name
        self.developer = developer
        self.price = price
        self.links = links
        
    def print_app(self):
        print("Name: " + self.name + 
                "\r\nDeveloper: " + self.developer + 
                "\r\nPrice: " + self.price + "\r\n")

In [5]:
crawler = AppCrawler('https://itunes.apple.com/ru/app/candy-crush-saga/id553834731?l=en&mt=8', 1)
crawler.crawl()

In [7]:
for app in crawler.apps:
    app.print_app()

Name: Candy Crush Saga
Developer: By King
Price: Free

Name: Juice Jam
Developer: By SGN
Price: Free

Name: Bejeweled Blitz
Developer: By PopCap
Price: Free

Name: Jewel Mania™
Developer: By Storm8 Studios
Price: Free

Name: Diamond Dash
Developer: By wooga
Price: Free

Name: Bubble Mania™ - Free Bubble Shooter
Developer: By Storm8 Studios
Price: Free



In [161]:
class YandexNewsCrawler:
    def __init__(self):
        '''self.rubr_dict = {"1politics":  "https://news.yandex.ru/politics.html?from=rubric",
                          "1society" :  "https://news.yandex.ru/society.html?from=rubric",
                          "1business":  "https://news.yandex.ru/business.html?from=rubric",
                          "1world":     "https://news.yandex.ru/world.html?from=rubric",
                          "sport":     "https://news.yandex.ru/sport.html?from=rubric",
                          "incident":  "https://news.yandex.ru/incident.html?from=rubric",
                          "1culture":   "https://news.yandex.ru/culture.html?from=rubric",
                          "science":   "https://news.yandex.ru/science.html?from=rubric",
                          "1computers": "https://news.yandex.ru/computers.html?from=rubric",
                          "1auto":      "https://news.yandex.ru/auto.html?from=rubric"}'''
        
        self.rubr_dict = {
                          "sport":     "https://news.yandex.ru/sport.html?from=rubric",
                          "incident":  "https://news.yandex.ru/incident.html?from=rubric",
                          "science":   "https://news.yandex.ru/science.html?from=rubric"
                         }
        
    def crawl(self):
        for rubr_name, rubr_url in self.rubr_dict.items():
            rubr = Rubric(rubr_name, rubr_url)
            rubr.crawl()
            print("Crawled ", rubr_name)
            texts = np.array(rubr.texts)
            np.save("./YandexNews10/" + rubr_name, texts)
            
        
class Rubric:
    def __init__(self, rubr_name, rubr_url):
        self.rubr_name = rubr_name
        self.rubr_url = rubr_url
        self.rubr_links = []
        self.group_links = []
        self.texts = []
    
    def crawl(self):
        start_time = time.time()
        self.rubr_links = self.parse_rubric_page(self.rubr_url)
        for link in self.rubr_links:
            article_links = self.parse_inside_chosen(link)
            self.group_links.append(article_links)
            for url in article_links:
                text = self.parse_article(url)
                self.texts.append(text)
                time.sleep(5)
        print("Crawling time: ", time.time() - start_time)

        
    def parse_rubric_page(self, link):
        start_page = requests.get(link)
        tree = html.fromstring(start_page.text)
        links = tree.xpath('//div[@class="story__content"]/h2/a/@href')
        #descrs = tree.xpath('//div[@class="story__text"]/text()')
        return links
    
    def parse_inside_chosen(self, link):
        base = "https://news.yandex.ru"
        start_page = requests.get(base + link)
        tree = html.fromstring(start_page.text)
        times = tree.xpath('//div[@class="story__list"]//*/div[@class="doc__time"]/text()')
        agencies = tree.xpath('//div[@class="story__list"]//*/div[@class="doc__agency"]/text()')
        #story_list = tree.xpath('//div[@class="story__tabs-panes tabs-panes i-bem"]//*/div[@class="story__list"]')
        article_links = tree.xpath('//div[@class="story__list"]//*/div[@class="doc__head"]/h2/a/@href')
        #print(story_list)
        return article_links[:1]
    
    def parse_article(self, link):
        article_page = requests.get(link)
        tree = html.fromstring(article_page.text)
        p_tags = tree.xpath('//p/text()')
        citates = tree.xpath('//blockquote/text()')
        h1_texts = tree.xpath('//h1/text()')
        h2_texts = tree.xpath('//h2/text()')
        h3_texts = tree.xpath('//h3/text()')
        article_text = p_tags + citates + h1_texts + h2_texts + h3_texts
        return prepare_text(' '.join(article_text))

    def prepare_text(self, text):
        return stem_file(text)
    
    def stem_file(text):
        text = re.sub(r"(\n)", " ", text.lower())
        text = re.split("[^а-я]", text)
        morph = pymorphy2.MorphAnalyzer()
        stemmed_text = []
        for word in text:
            if len(word) > 0:
                stemmed_text.append(morph.parse(word)[0].normal_form)
        stemmed_text = [word for word in stemmed_text if word not in stopwords.words("russian")]
        return stemmed_text

In [163]:
crawler = YandexNewsCrawler()
crawler.crawl()

Crawling time:  108.7225821018219
Crawled  computers
Crawling time:  101.08179903030396
Crawled  society
Crawling time:  114.44900393486023
Crawled  culture
Crawling time:  103.81476020812988
Crawled  world
Crawling time:  111.8690710067749
Crawled  business
Crawling time:  105.89873814582825
Crawled  auto
Crawling time:  105.34562516212463
Crawled  politics
Crawling time:  108.32536506652832
Crawled  sport
Crawling time:  108.7105770111084
Crawled  science
Crawling time:  102.34683895111084
Crawled  incident
